In [41]:
import pandas as pd 
import numpy as np 
import statsmodels.formula.api as smf

In [2]:
data = pd.read_csv('../Data/countyVLivEdu.csv')
data

,Unnamed: 0,case_id,state_x,isMetro,areaname,county,family_member_count,housing_cost,food_cost,transportation_cost,...,Col,state_y,county_name,totalvotes,DEMOCRAT,GREEN,LIBERTARIAN,OTHER,REPUBLICAN,Abv
0,0,1,AL,True,"Montgomery, AL MSA",Autauga,1p0c,8505.72876,3454.91712,10829.16876,...,28.131469,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368,AL
1,1,1,AL,True,"Montgomery, AL MSA",Autauga,1p1c,12067.50240,5091.70788,11588.19288,...,28.131469,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368,AL
2,2,1,AL,True,"Montgomery, AL MSA",Autauga,1p2c,12067.50240,7460.20308,12361.77720,...,28.131469,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368,AL
3,3,1,AL,True,"Montgomery, AL MSA",Autauga,1p3c,15257.15040,9952.23924,13452.18600,...,28.131469,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368,AL
4,4,1,AL,True,"Montgomery, AL MSA",Autauga,1p4c,15257.15040,12182.21400,13744.59840,...,28.131469,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22185,22185,3171,WY,False,"Weston County, WY",Weston,2p0c,8316.00000,6010.65672,14790.36240,...,18.397383,WYOMING,Weston,3560,0.101124,0.0,0.012921,0.013202,0.872753,WY
22186,22186,3171,WY,False,"Weston County, WY",Weston,2p1c,10956.00000,7480.86696,15611.58600,...,18.397383,WYOMING,Weston,3560,0.101124,0.0,0.012921,0.013202,0.872753,WY
22187,22187,3171,WY,False,"Weston County, WY",Weston,2p2c,10956.00000,9474.35748,16163.90280,...,18.397383,WYOMING,Weston,3560,0.101124,0.0,0.012921,0.013202,0.872753,WY
22188,22188,3171,WY,False,"Weston County, WY",Weston,2p3c,13632.00000,11567.46972,17118.38400,...,18.397383,WYOMING,Weston,3560,0.101124,0.0,0.012921,0.013202,0.872753,WY


In [17]:
#X = data[['median_family_income', 'REPUBLICAN', 'DEMOCRAT',
#        'noHS', 'HS',	'someCol', 'Col', 'family_member_count']]
#y = data['total_cost']

In [55]:

mod = smf.ols("total_cost ~ family_member_count +median_family_income + family_member_count : median_family_income + DEMOCRAT + REPUBLICAN + noHS + HS + someCol + Col", data=data).fit()
mod.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             total_cost   R-squared:                       0.880
Model:                            OLS   Adj. R-squared:                  0.880
Method:                 Least Squares   F-statistic:                     6776.
Date:                Sat, 09 Dec 2023   Prob (F-statistic):               0.00
Time:                        13:44:09   Log-Likelihood:            -2.3022e+05
No. Observations:               22190   AIC:                         4.605e+05
Df Residuals:                   22165   BIC:                         4.607e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                           56.3861      2.683     21.014      0.000      51.127      61.645
family_member_count[T.1p1c]                       2341.8144   1026.118      2.282      0.022     330.550    4353.079
family_member_count[T.1p2c]                       7482.2147   1026.118      7.292      0.000    5470.950    9493.479
family_member_count[T.1p3c]                       1.248e+04   1026.118     12.159      0.000    1.05e+04    1.45e+04
family_member_count[T.1p4c]                       1.712e+04   1026.118     16.685      0.000    1.51e+04    1.91e+04
family_member_count[T.2p0c]                       1.058e+04   1026.118     10.313      0.000    8571.190    1.26e+04
family_member_count[T.2p1c]                       1.525e+04   1026.118     14.863      0.000    1.32e+04    1.73e+04
family_member_count[T.2p2c]                       2.035e+04   1026.118     19.828      0.000    1.83e+04    2.24e+04
family_member_count[T.2p3c]                       2.453e+04   1026.118     23.903      0.000    2.25e+04    2.65e+04
family_member_count[T.2p4c]                       2.912e+04   1026.118     28.375      0.000    2.71e+04    3.11e+04
median_family_income                                 0.0573      0.011      5.255      0.000       0.036       0.079
family_member_count[T.1p1c]:median_family_income     0.2156      0.014     14.940      0.000       0.187       0.244
family_member_count[T.1p2c]:median_family_income     0.3363      0.014     23.307      0.000       0.308       0.365
family_member_count[T.1p3c]:median_family_income     0.4829      0.014     33.465      0.000       0.455       0.511
family_member_count[T.1p4c]:median_family_income     0.4980      0.014     34.511      0.000       0.470       0.526
family_member_count[T.2p0c]:median_family_income     0.0385      0.014      2.671      0.008       0.010       0.067
family_member_count[T.2p1c]:median_family_income     0.2157      0.014     14.945      0.000       0.187       0.244
family_member_count[T.2p2c]:median_family_income     0.3217      0.014     22.293      0.000       0.293       0.350
family_member_count[T.2p3c]:median_family_income     0.4574      0.014     31.695      0.000       0.429       0.486
family_member_count[T.2p4c]:median_family_income     0.4768      0.014     33.043      0.000       0.449       0.505
DEMOCRAT                                         -9.905e+04   6790.489    -14.586      0.000   -1.12e+05   -8.57e+04
REPUBLICAN                                       -1.141e+05   6715.618    -16.995      0.000   -1.27e+05   -1.01e+05
noHS                                              1438.4920     69.206     20.786      0.000    1302.844    1574.140
HS                                       

In [56]:
beginningtex = """\\documentclass{report}
\\usepackage{booktabs}
\\begin{document}"""
endtex = "\end{document}"

f = open('myreg.tex', 'w')
f.write(beginningtex)
f.write(mod.summary().as_latex())
f.write(endtex)
f.close()